In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
%cd /content/drive/MyDrive/YLP

/content/drive/MyDrive/YLP


In [74]:
from RKNet import RKNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
import os
import time
import datetime

In [50]:
batch_size = 250

shift = transforms.Normalize(
    mean=[0.5,0.5,0.5],
    std=[1.,1.,1.],
  )

# Load standard images for control purposes
transformTest = transforms.Compose(
    [transforms.ToTensor(),
     shift])


testSet = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transformTest)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batch_size,
                                          shuffle=False)

eps = 0.05

# Perturb testing images
hueTransform = transforms.Compose(
[lambda x: transforms.functional.adjust_hue(x,hue_factor=eps),
  transforms.ToTensor(),
  shift]
)

hueSet = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=hueTransform)
hueLoader = torch.utils.data.DataLoader(hueSet, batch_size=batch_size,
                                        shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [57]:
def imshow(img):
  img = img / 2 + 0.5
  npimg = img.cpu().numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

In [52]:
# Load trained network
PATH = os.path.join('results','current_pdenet.pt')
net = torch.load(PATH)
# Eval only needed
net.eval()

RKNet(
  (dynamicBlocks): ModuleList(
    (0): rk4(
      (controlLayers): ModuleList(
        (0): DoubleSymLayer(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (act): ReLU()
          (normLayer): TvNorm()
          (convt): ConvTranspose2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (1): DoubleSymLayer(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (act): ReLU()
          (normLayer): TvNorm()
          (convt): ConvTranspose2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (2): DoubleSymLayer(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (act): ReLU()
          (normLayer): TvNorm()
          (convt): ConvTranspose2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (3): DoubleSymLayer(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1

In [54]:
# Choose device and load model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

In [72]:
import csv

In [77]:

start = time.time()
sStartTime = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
file_name = 'norm_diff_parabolic' + sStartTime + '.csv'

csv_path = os.path.join('results',file_name)


header = ['eps','y0_diff','y_diff']
with open(csv_path,'w') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  

  # iterate over the mini-batches of normal and perturbed inputs together
  for data, data_tild in zip(testLoader,hueLoader):
    
    images = data[0]
    images_hue = data_tild[0]

    images = images.to(device)
    images_hue = images_hue.to(device)
    
    
    # Pass both batches of images through network
    outputs = net(images)
    outputs_hue = net(images_hue)
    # iterate through the mini-batch
    for i in range(len(images)):
      # Find initial values
      y0 = images[i]
      y0_tild = images_hue[i]
        
      # Find final values
      y = outputs[i]
      y_tild = outputs_hue[i]

      # Take norms and save
      norm_init = torch.linalg.norm(y0-y0_tild)
      norm_final = torch.linalg.norm(y-y_tild)
      writer.writerow([eps,norm_init.item(),norm_final.item()])

Files already downloaded and verified
